# Guided Project: Analyzing Startup Fundraising Deals from Crunchbase

## 1. Introduction

### 1.1 Data Loading

In [1]:
!pip install -r requirements.txt

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: Could not find a version that satisfies the requirement sqlite3 (from -r requirements.txt (line 2)) (from versions: none)
ERROR: No matching distribution found for sqlite3 (from -r requirements.txt (line 2))


In [2]:
# Import library for data loading
import pandas as pd

In this part, i am using pandas chunksize due to large dataset size (approximately 60 MB consist more than 50,000 rows)

In [3]:
data_chunks = pd.read_csv("data/crunchbase-investments.csv", chunksize = 5000, encoding='ISO-8859-1')

### 1.2 Each Column's Missing Value Counts

In [4]:
missing_val = []
memory_usg = []

#filling missing_val and memory_usg variable
for chunk in data_chunks :
    missing_val.append(chunk.isnull().sum()) 
    # Each column's missing value counts
    memory_usg.append(chunk.memory_usage(deep=True))
    # Each column's memory footprint

In [5]:
concat_mvc = pd.concat(missing_val)
missing_val_comb = concat_mvc.groupby(concat_mvc.index).sum().sort_values(ascending=False)
print('Missing value per column :')
print(missing_val_comb)

Missing value per column :
investor_category_code    50427
investor_state_code       16809
investor_city             12480
investor_country_code     12001
raised_amount_usd          3599
company_category_code       643
company_city                533
company_state_code          492
funding_round_type            3
funded_year                   3
funded_month                  3
funded_at                     3
funded_quarter                3
investor_name                 2
investor_permalink            2
investor_region               2
company_region                1
company_permalink             1
company_name                  1
company_country_code          1
dtype: int64


### 1.3 Memory Footprint

In [6]:
# Each column's memory footprint
concat_mu = pd.concat(memory_usg)
memory_usg_comb = concat_mu.groupby(concat_mu.index).sum().sort_values(ascending=False)
print('Memory usage each column :')
print(memory_usg_comb.drop('Index'))

# The total memory footprint of all of the chunks combined
print('\nMemory usage Sum = {} KB'.format(memory_usg_comb.sum()))

Memory usage each column :
investor_permalink        4980548
company_permalink         4057788
investor_name             3915666
company_name              3591326
funded_at                 3542185
company_city              3505906
company_category_code     3421104
company_region            3411565
funding_round_type        3410707
investor_region           3396281
funded_month              3383584
funded_quarter            3383584
company_country_code      3172176
company_state_code        3106051
investor_city             2885083
investor_country_code     2647292
investor_state_code       2476607
investor_category_code     622424
raised_amount_usd          422960
funded_year                422960
dtype: int64

Memory usage Sum = 59756717 KB


### 1.4 Eliminating Column(s) that aren't Useful for Analysis

In [7]:
# print sample from random chunk
chunk.sample(10)

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50159,/company/cardpool,Cardpool,web,USA,CA,SF Bay,San Francisco,/person/nils-johnson,Nils Johnson,NaN,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,NaN
52420,/company/storenvy,Storenvy,ecommerce,USA,CA,SF Bay,San Francisco,/person/tom-rosedale,Tom Rosedale,NaN,NaN,NaN,unknown,NaN,series-a,2011-01-27,2011-01,2011-Q1,2011,1525000.0
52325,/company/checkpoints,CheckPoints,mobile,USA,CA,Los Angeles,Los Angeles,/person/todd-dipaola,Todd Dipaola,NaN,NaN,NaN,unknown,NaN,series-a,2010-05-01,2010-05,2010-Q2,2010,1000000.0
51979,/company/justme,just.me,mobile,USA,CA,SF Bay,Palo Alto,/person/steve-mcarthur,Steve McArthur,NaN,NaN,NaN,unknown,NaN,angel,2011-07-30,2011-07,2011-Q3,2011,500000.0
51846,/company/myminilife,MyMiniLife,games_video,USA,CA,SF Bay,Palo Alto,/person/sizhao-yang,Sizhao Yang,NaN,NaN,NaN,unknown,NaN,angel,2006-11-15,2006-11,2006-Q4,2006,10000.0
50511,/company/adility,Adility,advertising,USA,CA,SF Bay,San Francisco,/person/peter-bordes,Peter Bordes,NaN,NaN,NaN,unknown,NaN,series-a,2011-09-06,2011-09,2011-Q3,2011,NaN
51301,/company/circle-inc,Circle Inc,social,USA,CA,SF Bay,Palo Alto,/person/ron-conway,Ron Conway,NaN,NaN,NaN,unknown,NaN,angel,2011-01-01,2011-01,2011-Q1,2011,1000000.0
50813,/company/t5-data-centers,T5 Data Centers,security,USA,GA,Atlanta,Atlanta,/person/raymond-james,Raymond James,NaN,NaN,NaN,unknown,NaN,other,2013-05-14,2013-05,2013-Q2,2013,113000000.0
50541,/company/kickstarter,Kickstarter,finance,USA,NY,New York,New York,/person/peter-hershberg,Peter Hershberg,NaN,NaN,NaN,unknown,NaN,venture,2011-03-18,2011-03,2011-Q1,2011,10000000.0
50929,/company/hoteltonight,HotelTonight,mobile,USA,CA,SF Bay,San Francisco,/person/rich-barton-2,Rich Barton,NaN,NaN,NaN,unknown,NaN,series-a,2011-05-10,2011-05,2011-Q2,2011,3585000.0


In [8]:
drop_cols = ['investor_permalink','company_permalink','investor_category_code','funded_year','funded_month','funded_quarter']

Explanation :
1. Permalink columns ('investor_permalink' and 'company_permalink') can't be used for analysis
2. 'investor_category_code" is column with highest missing value, therefore can't be used for analysis
3. 'funded_year', 'funded_month' and 'funded_quarter' is considered redundant and can be substituted by 'funded_at'

## 2. Selecting Data Types

### 2.1 Identify the types for each column per chunk

In [9]:
data_chunks = pd.read_csv("data/crunchbase-investments.csv", chunksize = 5000, encoding='ISO-8859-1')
col_types = {}

for chunk in data_chunks:
    chunk.drop(columns=drop_cols, inplace = True)
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))

In [10]:
col_types_df = pd.DataFrame(col_types, columns = chunk.columns)
col_types_df.index.name = "chunk"

In [11]:
col_types_df

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,raised_amount_usd
chunk,,,,,,,,,,,,,,
0,object,object,object,object,object,object,object,object,object,object,object,object,object,float64
1,object,object,object,object,object,object,object,object,object,object,object,object,object,float64
2,object,object,object,object,object,object,object,object,object,object,object,object,object,float64
3,object,object,object,object,object,object,object,object,object,object,object,object,object,float64
4,object,object,object,object,object,object,object,object,object,object,object,object,object,float64
5,object,object,object,object,object,object,object,object,object,object,object,object,object,float64
6,object,object,object,object,object,object,object,object,object,object,object,object,object,float64
7,object,object,object,object,object,object,object,object,object,object,object,object,object,float64
8,object,object,object,object,object,object,object,object,object,object,object,object,object,float64


In [12]:
# Identify the unique types for each columns
uniq_col_types = {}
for k,v in col_types.items():
    uniq_col_types[k] = set(col_types[k])
uniq_col_types

{'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_name': {'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_state_code': {'float64', 'object'},
 'investor_region': {'object'},
 'investor_city': {'float64', 'object'},
 'funding_round_type': {'object'},
 'funded_at': {'object'},
 'raised_amount_usd': {'float64'}}

### 2.2 Find more space efficient types for every columns

#### 2.2.1 Find Columns that can be Converted into Category and Datetime 

In [13]:
# Find the Number of Unique Values in Each Non-Numeric Column
df = pd.read_csv("data/crunchbase-investments.csv", encoding='ISO-8859-1', low_memory=False)
df.drop(columns=drop_cols, inplace = True)

print("\nMany unique values in each columns:")
for col in df.columns:
    print("{} : {}".format(col,len(df[col].unique())))


Many unique values in each columns:
company_name : 11574
company_category_code : 44
company_country_code : 3
company_state_code : 51
company_region : 547
company_city : 1230
investor_name : 10466
investor_country_code : 73
investor_state_code : 51
investor_region : 586
investor_city : 991
funding_round_type : 10
funded_at : 2809
raised_amount_usd : 1459


If the amount of unique strings in column cover less than half the total amount of strings, the column is suitable to convert into pandas category column type. Keep in mind that column with category data type restricts adding unknown values to that column.

In [14]:
def optimize_objects(df,date_time_column):
    for col in df.select_dtypes(include=['object']):
        if col not in date_time_column:
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                df[col] = df[col].astype('category')
        else:
            df[col] = pd.to_datetime(df[col])
    return df

## 3. The Total Memory Footprint After Data Cleaning

In [15]:
data_chunks = pd.read_csv("data/crunchbase-investments.csv", chunksize = 5000, encoding='ISO-8859-1')

memory_usg = []

#filling missing_val and memory_usg variable
for chunk in data_chunks :
    chunk.drop(columns=drop_cols, inplace = True)
    chunk = optimize_objects(chunk,['funded_at'])
    memory_usg.append(chunk.memory_usage(deep=True))
    # Each column's memory footprint

In [16]:
# Each column's memory footprint
concat_mu = pd.concat(memory_usg)
memory_usg_comb = concat_mu.groupby(concat_mu.index).sum().sort_values(ascending=False)
print('Memory usage each column :')
print(memory_usg_comb.drop('Index'))

# The total memory footprint of all of the chunks combined
print('\nMemory usage Sum = {} KB'.format(memory_usg_comb.sum()))

Memory usage each column :
company_name             3489096
investor_name            1221123
company_city              654349
raised_amount_usd         422960
funded_at                 422960
investor_city             370095
company_region            332777
investor_region           227785
investor_state_code       138773
investor_country_code     138080
company_category_code      96448
company_state_code         96101
funding_round_type         62126
company_country_code       54474
dtype: int64

Memory usage Sum = 7728067 KB


After dropping six columns and converting several columns into categorical and datetime datatype, The total memory usage have dropped from 59.76 MB to 7.72 MB, improving over 87%. So eliminating and converting columns was definitely worth it.

## 4. Loading Chunks Into SQLite

In [17]:
import sqlite3
conn = sqlite3.connect('data/crunchbase.db')
data_chunks = pd.read_csv('data/crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

# uncomment two lines below if you want to renew (not append) 'investments' table
cur = conn.cursor()
cur.execute("""DROP TABLE investments;""")

for chunk in data_chunks:
    chunk.drop(columns=drop_cols, inplace = True)
    chunk = optimize_objects(chunk,['funded_at'])
    chunk.to_sql("investments", conn, if_exists='append', index=False)
    
conn.commit()
conn.close()

In [18]:
!wc data/crunchbase.db

   11299   589499 14204928 data/crunchbase.db


## 5. Data Exploration and Analysis Using SQLite

In [19]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data/crunchbase.db')
cur = conn.cursor()
query = ("""SELECT * FROM investments """)
pd.read_sql(query, conn)

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,raised_amount_usd
0,AdverCar,advertising,USA,CA,SF Bay,San Francisco,1-800-FLOWERS.COM,USA,NY,New York,New York,series-a,2012-10-30 00:00:00,2000000.0
1,LaunchGram,news,USA,CA,SF Bay,Mountain View,10Xelerator,USA,OH,Columbus,Columbus,other,2012-01-23 00:00:00,20000.0
2,uTaP,messaging,USA,None,United States - Other,None,10Xelerator,USA,OH,Columbus,Columbus,other,2012-01-01 00:00:00,20000.0
3,ZoopShop,software,USA,OH,Columbus,columbus,10Xelerator,USA,OH,Columbus,Columbus,angel,2012-02-15 00:00:00,20000.0
4,eFuneral,web,USA,OH,Cleveland,Cleveland,10Xelerator,USA,OH,Columbus,Columbus,other,2011-09-08 00:00:00,20000.0
5,Tackk,web,USA,OH,Cleveland,Cleveland,10Xelerator,USA,OH,Columbus,Columbus,other,2012-02-01 00:00:00,20000.0
6,Acclaimd,analytics,USA,OH,Columbus,Columbus,10Xelerator,USA,OH,Columbus,Columbus,angel,2012-06-01 00:00:00,20000.0
7,Acclaimd,analytics,USA,OH,Columbus,Columbus,10Xelerator,USA,OH,Columbus,Columbus,angel,2012-08-07 00:00:00,70000.0
8,ToVieFor,ecommerce,USA,NY,New York,New York,2010 NYU Stern Business Plan Competition,None,None,unknown,None,angel,2010-04-01 00:00:00,75000.0
9,OHK Labs,sports,USA,FL,Palm Beach,Boca Raton,22Hundred Group,None,None,unknown,None,angel,2011-09-01 00:00:00,100000.0


### 5.1 Proportion of the total amount of funds raised

#### 5.1.1 Top 10% Company

In [20]:
query = ("""SELECT SUM(sum_all)
FROM (
    SELECT SUM(raised_amount_usd) AS sum_all
    FROM investments
    GROUP BY company_name
    ORDER BY SUM(raised_amount_usd) DESC 
    LIMIT 1157
) AS T""")
pd.read_sql(query, conn)

,SUM(sum_all)
0,4.576313e+11


#### 5.1.2 Top 1% Company

In [21]:
query = ("""SELECT SUM(sum_all)
FROM (
    SELECT SUM(raised_amount_usd) AS sum_all
    FROM investments
    GROUP BY company_name
    ORDER BY SUM(raised_amount_usd) DESC 
    LIMIT 116
) AS T""")
pd.read_sql(query, conn)

,SUM(sum_all)
0,1.793735e+11


#### 5.1.3 Bottom 10% Company

In [22]:
query = ("""SELECT SUM(sum_all)
FROM (
    SELECT SUM(raised_amount_usd) AS sum_all
    FROM investments
    WHERE raised_amount_usd iS NOT NULL
    GROUP BY company_name
    ORDER BY SUM(raised_amount_usd) ASC 
    LIMIT 1157
) AS T""")
pd.read_sql(query, conn)

,SUM(sum_all)
0,252174228.0


#### 5.1.4 Bottom 1% Company

In [23]:
query = ("""SELECT SUM(sum_all)
FROM (
    SELECT SUM(raised_amount_usd) AS sum_all
    FROM investments
    WHERE raised_amount_usd iS NOT NULL
    GROUP BY company_name
    ORDER BY SUM(raised_amount_usd) ASC 
    LIMIT 116
) AS T""")
pd.read_sql(query, conn)

,SUM(sum_all)
0,1741100.0


### 5.2 Which category of company attracted the most investments?

In [24]:
query = ("""SELECT company_category_code, SUM(raised_amount_usd)
FROM investments 
GROUP BY company_category_code 
ORDER BY SUM(raised_amount_usd) DESC 
LIMIT 10""")
pd.read_sql(query, conn)

,company_category_code,SUM(raised_amount_usd)
0,biotech,1.103964e+11
1,software,7.308452e+10
2,mobile,6.477738e+10
3,cleantech,5.270523e+10
4,enterprise,4.586093e+10
5,web,4.014326e+10
6,medical,2.536711e+10
7,advertising,2.507666e+10
8,ecommerce,2.256722e+10
9,network_hosting,2.241968e+10


### 5.3 Which investor contributed the most money (across all startups)?

In [25]:
query = ("""SELECT DISTINCT investor_name, SUM(raised_amount_usd)
FROM investments
GROUP BY investor_name
ORDER BY SUM(raised_amount_usd) DESC 
LIMIT 10""")
pd.read_sql(query, conn)

,investor_name,SUM(raised_amount_usd)
0,Kleiner Perkins Caufield & Byers,1.121783e+10
1,New Enterprise Associates,9.692542e+09
2,Accel Partners,6.472126e+09
3,Goldman Sachs,6.375459e+09
4,Sequoia Capital,6.039402e+09
5,Intel,5.969200e+09
6,Google,5.808800e+09
7,Time Warner,5.730000e+09
8,Comcast,5.669000e+09
9,Greylock Partners,4.960983e+09


### 5.4 Which investors contributed the most money per startup?

In [26]:
query = ("""SELECT investor_name, company_name, SUM(raised_amount_usd)
FROM investments
GROUP BY company_name
ORDER BY SUM(raised_amount_usd) DESC
LIMIT 10""")
pd.read_sql(query, conn)

,investor_name,company_name,SUM(raised_amount_usd)
0,BrightHouse,Clearwire,2.968000e+10
1,Accel Partners,Groupon,1.018540e+10
2,Beck Energy,Nanosolar,4.505000e+09
3,Microsoft,Facebook,4.154100e+09
4,Google,SurveyMonkey,3.250000e+09
5,Google,Zynga,2.886013e+09
6,A123 Systems,Fisker Automotive,2.788000e+09
7,Y Combinator,Dropbox,2.764400e+09
8,Amazon,LivingSocial,2.685000e+09
9,Marlin Equity Partners,sigmacare,2.600000e+09


### 5.5 Which funding round was the most popular? Which was the least popular?

In [27]:
query = ("""SELECT funding_round_type, SUM(raised_amount_usd) FROM investments 
GROUP BY funding_round_type 
ORDER BY SUM(raised_amount_usd) DESC""")
pd.read_sql(query, conn)

,funding_round_type,SUM(raised_amount_usd)
0,series-c+,2.657535e+11
1,venture,1.305565e+11
2,series-b,1.283268e+11
3,series-a,8.654215e+10
4,post-ipo,3.091760e+10
5,other,1.850726e+10
6,private-equity,1.615988e+10
7,angel,4.962075e+09
8,crowdfunding,6.491500e+06
9,None,NaN


In [28]:
# Close connection
conn.close()